In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os

import pandas as pd

from collections import Counter


In [2]:
def Img_into_volt(Img, selection_mask, volt_scale = 1):
    
    Img_sub = np.empty((16, (Img.shape[0]-3)*(Img.shape[1]-3)))
    for n in range(Img.shape[0]-3):
        for m in range(Img.shape[1]-3):
            Img_sub[:,n*10+m] = Img[n:n+4,m:m+4].flatten()*volt_scale

    
    rows = np.where(selection_mask.sum(axis=1)!=0)[0]
    output = np.zeros((32,100))
    output[rows,:] = Img_sub
        
    return output


In [3]:
def findRowCols(data, rowselectionsize=16, colselectionsize=10, searchingsize=18):
    data_sorted = np.sort(data, axis=0)
    index_sorted = np.argsort(data, axis=0)

    delta = np.empty((1, data_sorted.shape[1]))
    for i in np.arange(data_sorted.shape[0] - rowselectionsize + 1):
        delta = np.vstack((delta, data_sorted[i + rowselectionsize - 1, :] - data_sorted[i, :]))
    delta = np.delete(delta, 0, axis=0)

    res = np.empty((rowselectionsize, data.shape[1]))  # res is row combinations of rows index
    for col, start in zip(range(data.shape[1]), np.argmin(delta, axis=0)):
        res[:, col] = index_sorted[start:start + rowselectionsize, col]

    row_cnt = Counter(res.reshape(-1).astype(int))
    top_searchingsize_rows = [a for (a, _) in row_cnt.most_common(data.shape[1])[:searchingsize]]

    maxdelta = {}
    cols = {}
    for rows in itertools.combinations(top_searchingsize_rows, rowselectionsize):
        top_16_data = data[np.array(rows)]
        col_delta = pd.Series(np.ptp(top_16_data, axis=0))
        x = col_delta.nsmallest(colselectionsize)
        cols[rows] = list(x.index.values)
        maxdelta[rows] = x.iloc[-1]

    Maxdelta = pd.Series(maxdelta).nsmallest(1).values[0]
    Rows = list(list(pd.Series(maxdelta).nsmallest(1).keys())[0])
    Columns = cols[tuple(Rows)]
    return Maxdelta, np.sort(Rows), np.sort(Columns)


def make_selection_mask(data, rowselectionsize = 16, colselectionsize = 10, searchingsize = 21):
    d, R, C = findRowCols(data, rowselectionsize = rowselectionsize, colselectionsize = colselectionsize, searchingsize = searchingsize)

    selection = np.zeros((32,32))

    selection[R[:,np.newaxis],C] = 1

    np.savetxt("C:\\Users\\Houk\\Dropbox\\ISNeuroData\\2di_mb_py_probe\\selection_mask.csv", selection,  delimiter=",")

    
    return selection


In [4]:
def get_target_scale(selection_mask, weights):
    def index_gen():
        for i in range(32):
            for j in range(32):
                if selection_mask[i, j] != 0:
                    yield (i, j)

    target_scale = np.zeros((32, 32))
    select_index = index_gen()

    for i in range(len(weights)):
        for j in range(len(weights[0])):
            target_scale[next(select_index)] = weights[i, j]

    return target_scale

In [5]:
data = np.genfromtxt("Imager Mnist/initial_data.csv", delimiter=",")
data

array([[ 1.40285626e+02,  1.92356293e+02,  1.91450302e+02, ...,
         2.21586439e+02,  2.59495029e+02,  1.87397182e+02],
       [ 2.15673652e+02,  2.61402379e+02,  1.76525284e+02, ...,
         3.42274039e+02,  5.14698519e+02,  4.28533963e+02],
       [ 7.41959326e+01,  1.40905515e+02,  1.77622011e+02, ...,
         3.17526166e+02,  3.12996209e+02,  1.34658942e+02],
       ...,
       [ 2.73323320e+02,  5.41258374e+02,  4.92191784e+02, ...,
         7.34472975e+02,  6.91939060e+02,  4.70352621e+02],
       [ 2.99644756e+02,  8.87013328e+02,  5.69677896e+02, ...,
         7.79152660e+02,  1.07984646e+03,  5.09167203e+02],
       [-1.04904275e+00,  4.57478006e+02,  3.62110483e+02, ...,
         5.78499392e+02,  6.60467778e+02,  5.41449110e+02]])

In [6]:
selection_mask = make_selection_mask(data, rowselectionsize = 16, colselectionsize = 10, searchingsize = 21)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Houk\\Dropbox\\ISNeuroData\\2di_mb_py_probe\\selection_mask.csv'

In [ ]:
plt.imshow(selection_mask)

In [ ]:
Img = np.genfromtxt("Imager Mnist/0.csv", delimiter=",")

image_encoded = Img_into_volt(Img, selection_mask, volt_scale = 0.3 )
image_encoded
np.savetxt("C:\\Users\\Houk\\Dropbox\\ISNeuroData\\2di_mb_py_probe\\image_encoded.csv", image_encoded,  delimiter=",")

In [37]:
image_encoded.min()

0.005